In [21]:
from __future__ import division

import numpy as np
from gensim.models import Word2Vec 
import scipy.stats as stat
from scipy.special import gamma
import scipy
from numpy import pi
from numba import jit, vectorize, float32, float64, 
import time
import line_profiler
import cProfile
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)
%load_ext line_profiler
%load_ext autoreload


In [6]:
wordvecs = Word2Vec.load_word2vec_format(
    "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt", binary=False)

In [ ]:
import FastGaussianLDA
%aimport FastGaussianLDA
%autoreload 1
# import GaussianLDA
# reload(Gauss_LDA)
# reload(GaussianLDA)
start  = time.time()

corpus = ['apple banana mango melon',
 'barrel cannon gun war',
 'democrat republican senate politics']
corpus2 = ["apple orange mango melon ", "canvas art mural paint painting ", "pineapple kiwi grape strawberry ",
              "picture frame picasso sculpture art ", "coconut guava blueberry blackberry ", "statue monument art artist "]
corpus2 = [sent * 5 for sent in corpus2]*4
wordvec_fileapth = "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt"
g = FastGaussianLDA.Gauss_LDA(3, corpus2, word_vector_model=wordvecs)
g.fit(100)
print (time.time()-start)

Done processing corpus with 24 documents
There are 24 words that could be convereted to word vectors in your corpus 
There are 0 words that could NOT be converted to word vectors
Intialization complete
Starting fit
0 iterations complete
1 iterations complete
2 iterations complete
3 iterations complete
4 iterations complete
5 iterations complete
6 iterations complete
7 iterations complete
8 iterations complete
9 iterations complete
10 iterations complete
11 iterations complete
12 iterations complete
13 iterations complete
14 iterations complete
15 iterations complete
16 iterations complete
17 iterations complete
18 iterations complete
19 iterations complete
20 iterations complete
21 iterations complete
22 iterations complete
23 iterations complete
24 iterations complete
25 iterations complete
26 iterations complete
27 iterations complete
28 iterations complete
29 iterations complete
30 iterations complete

In [187]:
IPython.get_ipython().

In [173]:
IPython.utils.sysinfo.get_sys_info()

{'commit_hash': u'b573435',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/Users/michael/anaconda/lib/python2.7/site-packages/IPython',
 'ipython_version': '4.1.2',
 'os_name': 'posix',
 'platform': 'Darwin-14.5.0-x86_64-i386-64bit',
 'sys_executable': '/Users/michael/anaconda/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '2.7.11 |Anaconda 2.3.0 (x86_64)| (default, Dec  6 2015, 18:57:58) \n[GCC 4.2.1 (Apple Inc. build 5577)]'}

In [175]:
IPython.

'i386'

old time was 237 secs on 30 iterations without NUMBA

In [23]:
mean1 = g.topic_params[0]["Topic Mean"]; mean2 = g.topic_params[1]["Topic Mean"]; mean3 = g.topic_params[2]["Topic Mean"]
wordvecs.most_similar(positive=[mean1])

[(u'sculpture', 0.8271721601486206),
 (u'painting', 0.8245132565498352),
 (u'art', 0.803730845451355),
 (u'collection', 0.7746897339820862),
 (u'garden', 0.7652238011360168),
 (u'famous', 0.7629948854446411),
 (u'works', 0.7435003519058228),
 (u'paintings', 0.7393991947174072),
 (u'sculptures', 0.7150182723999023),
 (u'glass', 0.7109430432319641)]

In [24]:
wordvecs.most_similar(positive=[mean2])

[(u'colored', 0.8344143629074097),
 (u'painted', 0.8143922090530396),
 (u'canvas', 0.8039060831069946),
 (u'paint', 0.8033887147903442),
 (u'green', 0.8015591502189636),
 (u'pink', 0.781296968460083),
 (u'shaped', 0.7649050951004028),
 (u'shiny', 0.7583204507827759),
 (u'piece', 0.7541706562042236),
 (u'bright', 0.7407790422439575)]

In [25]:
wordvecs.most_similar(positive=[mean3])

[(u'mango', 0.8706889152526855),
 (u'pineapple', 0.8398306369781494),
 (u'coconut', 0.8316406011581421),
 (u'peach', 0.8135504722595215),
 (u'fruit', 0.8121268153190613),
 (u'apricot', 0.7849801778793335),
 (u'juice', 0.7718098163604736),
 (u'plum', 0.7694075107574463),
 (u'banana', 0.7686874866485596),
 (u'cherry', 0.7675427198410034)]

In [ ]:
newdoc = "fruit apple orange mango taco painting sculpture book art artist picasso"

In [ ]:
def myfunc(mat):
    np.linalg.det(mat)
    return np.linalg.inv(mat)
    
%prun myfunc(np.arange(100).reshape(10,10))

In [6]:
a = None
if a: print 'yes'

### Expirmenting with Numba accelerations:
- Using numpy ufuncs

In [31]:
50**2

2500

In [6]:
import numpy as np

In [41]:
a = np.cov(np.arange(50.0**2, dtype=np.float32).reshape(50,50))
a = np.eye(50.)*4
# scipy.linalg.cholesky(a, lower=True, overwrite_a=True)
x = np.arange(50)

In [110]:
from numba import generated_jit, vectorize

- Using python operators

In [24]:
@jit((float32[:,:]))
def chol_downdate(L, X):
        """
        Cholesky Rank 1 Update
        :param L: Lower triangle matrix from Cholesky Decomposition
        :param X: Word Vector with same column dimensionality as L
        :return: updated lower triangle matrix
        """
        # assert L.shape[1] == X.shape[0]
        # choldowndate(L.T, X)
        L_c = np.copy(L) # in-place computations are faster
        for k in range(X.shape[0]):
            r = np.sqrt(L_c[k, k]**2 - X[k]**2)
            c = r / L_c[k, k]
            s = X[k] / L_c[k, k]
            L_c[k, k] = r

            for i in range(k+1, X.shape[0]):
                L_c[i, k] = (L_c[i, k] - (s * X[i])) / c
                X[i] = (c * X[i]) - (s * L_c[i, k])
        if np.isnan(r) or np.isinf(r):
            print "YOU GOT Nans or infs: learn to code better shmuck"
            return L # good reason for making copy - return if downdate becomes unstable
        return L_c

AttributeError: 'Array' object has no attribute '__defaults__'

In [139]:
%%timeit
chol_downdate(a, x)

The slowest run took 5.50 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 14.2 µs per loop


- Using numpy ufuncs is SLOWER

In [144]:
@jit
def chol_downdate(L, X):
        """
        Cholesky Rank 1 Update
        :param L: Lower triangle matrix from Cholesky Decomposition
        :param X: Word Vector with same column dimensionality as L
        :return: updated lower triangle matrix
        """
        assert L.shape[1] == X.shape[0]
        # choldowndate(L.T, X)
        L_c = np.copy(L) # in-place computations are faster
        for k in range(X.shape[0]):
            r = np.sqrt(np.power(L_c[k, k],2) - np.power(X[k], 2))
            c = r / L_c[k, k]
            s = X[k] / L_c[k, k]
            L_c[k, k] = r

            for i in range(k+1, X.shape[0]):
                L_c[i, k] = (L_c[i, k] - (s * X[i])) / c
                X[i] = (c * X[i]) - (s * L_c[i, k])
        if np.isnan(r) or np.isinf(r):
            print "YOU GOT Nans or infs: learn to code better shmuck"
            return L # good reason for making copy - return if downdate becomes unstable
        return L_c

In [147]:
%%timeit
chol_downdate(a, x)

The slowest run took 4.57 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 18.2 µs per loop


In [42]:
b = np.array([1, 1, 2,2,2,1])

In [43]:
a[b].shape

(6, 50)

In [44]:
a[[1,1],[1,1]]

array([ 4.,  4.])

In [30]:
a = set([1, 2, 3, 4]); b = set([3, 4, 5, 6]); aa = a.copy()
lsa = [a, aa]

In [45]:
np.flatiter?

In [32]:
c = [i&b for i in lsa]

In [39]:

d = a&b
dd = list(d)

In [37]:
d.symmetric_difference?

In [47]:
d = [b, b,b]

In [51]:
np.array(d).shape

(3, 6)

In [53]:
np.vstack(d).shape

(3, 6)

In [54]:
np.array?
